In [1]:
import sqlite3
import pandas as pd

In [2]:
conn=sqlite3.connect('testFlask\db.sqlite')
c = conn.cursor()

In [3]:
df = pd.read_csv('parser/users.csv', sep=';')[['id', 'email', 'password', 'surname', 'name']].astype('str')
c.executemany("INSERT INTO user VALUES(?,?,?,?,?)", df.values)

In [4]:
df = pd.read_csv('parser/rooms.csv', sep=';')[['id', 'name', 'type', 'x', 'y', 'floor']].astype('str')
c.executemany("INSERT INTO room VALUES(?,?,?,?,?,?)", df.values)

In [5]:
df = pd.read_csv('parser/places.csv', sep=';')[['id', 'number', 'user_id', 'room_id']].astype('str')
c.executemany("INSERT INTO place VALUES(?,?,?,?)", df.values)

In [6]:
c.close()
conn.commit()
conn.close()

id         int32
number     int32
user_id    int32
room_id    int32
dtype: object

In [ ]:
c.execute('select * from room')
c.fetchall()